In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

#### Загружаем очищенные данные

In [2]:
data19=pd.read_csv('df2019.csv',index_col=0)
data20=pd.read_csv('df2020.csv',index_col=0)
data21=pd.read_csv('df2021.csv',index_col=0)

data19.head(1)

,Наименование ДП,"2016, Нематериальные активы","2017, Нематериальные активы","2018, Нематериальные активы","2016, Основные средства","2017, Основные средства","2018, Основные средства","2016, Внеоборотные активы","2017, Внеоборотные активы","2018, Внеоборотные активы","2016, Дебиторская задолженность","2017, Дебиторская задолженность","2018, Дебиторская задолженность","2016, Оборотные активы","2017, Оборотные активы","2018, Оборотные активы","2016, Уставный капитал","2017, Уставный капитал","2018, Уставный капитал","2016, Капитал и резервы","2017, Капитал и резервы","2018, Капитал и резервы","2016, Заёмные средства (долгосрочные)","2017, Заёмные средства (долгосрочные)","2018, Заёмные средства (долгосрочные)","2016, Долгосрочные обязательства","2017, Долгосрочные обязательства","2018, Долгосрочные обязательства","2016, Заёмные средства (краткосрочные)","2017, Заёмные средства (краткосрочные)","2018, Заёмные средства (краткосрочные)","2016, Кредиторская задолженность","2017, Кредиторская задолженность","2018, Кредиторская задолженность","2016, Краткосрочные обязательства","2017, Краткосрочные обязательства","2018, Краткосрочные обязательства","2016, Выручка","2017, Выручка","2018, Выручка","2016, Себестоимость продаж","2017, Себестоимость продаж","2018, Себестоимость продаж","2016, Прибыль (убыток) до налогообложения","2017, Прибыль (убыток) до налогообложения","2018, Прибыль (убыток) до налогообложения","2016, Прибыль (убыток) от продажи","2017, Прибыль (убыток) от продажи","2018, Прибыль (убыток) от продажи","Макс. ПДЗ за 2019 год, дней"
0,0,2.895541e+06,6.245860e+06,9.050955e+06,1.889724e+09,1.803724e+09,2.145239e+09,1.928685e+09,1.880842e+09,2.440078e+09,3.583766e+09,3.683359e+09,3.475461e+09,5.159380e+09,5.897326e+09,6.077448e+09,629400000.0,629400000.0,629400000.0,2.673020e+09,2.971017e+09,2.289274e+09,0.0,0.0,0.0,2.498777e+08,2.589732e+08,2.712210e+08,0.0,0.0,3.923917e+08,4.080056e+09,4.452922e+09,5.476906e+09,4.165168e+09,4.548178e+09,5.957031e+09,2.667698e+10,4.162693e+10,3.703952e+10,2.156814e+10,3.335443e+10,3.438313e+10,3.179697e+09,6.167667e+09,7.461828e+08,3.280355e+09,6.200120e+09,8.716191e+08,0


### Генерация

#### Добавляем нужные столбцы

In [3]:
def other_columns(data,year,lags=3):
    
    for i in range(1,lags+1):
        data[str(year-i)+', '+'Активы']=data[str(year-i)+', '+'Оборотные активы']+data[str(year-i)+', '+'Внеоборотные активы']
        data[str(year-i)+', '+'Собственные оборотные средства']=data[str(year-i)+', '+'Оборотные активы']-data[str(year-i)+', '+'Краткосрочные обязательства']
        data[str(year-i)+', '+'Заёмные средства']=data[str(year-i)+', '+'Заёмные средства (краткосрочные)']+data[str(year-i)+', '+'Заёмные средства (долгосрочные)']
        data[str(year-i)+', '+'Чистые активы']=data[str(year-i)+', '+'Активы']-data[str(year-i)+', '+'Заёмные средства']
   
    return data

data19=other_columns(data19,year=2019)
data20=other_columns(data20,year=2020)
data21=other_columns(data21,year=2021)

#### Оставляем те компании, у которых активы не равны нулю

In [4]:
data19=data19[data19['2018, Активы']!=0].reset_index().drop('index',axis=1)
data20=data20[data20['2019, Активы']!=0].reset_index().drop('index',axis=1)
data21=data21[data21['2020, Активы']!=0].reset_index().drop('index',axis=1)

#### Убираем пробелы в начале и в конце строк

In [5]:
for i in range(len(data19.columns)):
    data19=data19.rename({data19.columns[i]: data19.columns[i].rstrip()},axis=1)
    data20=data20.rename({data20.columns[i]: data20.columns[i].rstrip()},axis=1)

#### Генерируем экономические коэффициенты

Подробнее: https://1fin.ru/?id=311

In [6]:
def feature_generation(data,year,lags=3):
    # Создаём новый датафрейм
    df_new=pd.DataFrame()
    df_new['Наименование ДП']=data['Наименование ДП']
    # Генерим новые признаки
    for i in range(1,lags+1):
        ### 1. Коэффициенты ликвидности
        df_new[str(i)+', '+'cf_likv1']=data[str(year-i)+', '+'Оборотные активы']/data[str(year-i)+', '+'Краткосрочные обязательства'] # Коэффициент текущей ликвидности
        df_new[str(i)+', '+'cf_likv2']=data[str(year-i)+', '+'Оборотные активы']/(
                                            data[str(year-i)+', '+'Краткосрочные обязательства']+data[str(year-i)+', '+'Долгосрочные обязательства']) # Коэффициент общей ликвидности
        df_new[str(i)+', '+'cf_likv3']=data[str(year-i)+', '+'Оборотные активы']/(
                                            data[str(year-i)+', '+'Краткосрочные обязательства']+data[str(year-i)+', '+'Кредиторская задолженность']) # Общий коэффициент покрытия
        ### 2. Коэффициенты оборачиваемости
        df_new[str(i)+', '+'cf_ob1']=data[str(year-i)+', '+'Выручка']/data[str(year-i)+', '+'Оборотные активы'] # Коэффициент оборачиваемости оборотных средств
        df_new[str(i)+', '+'cf_ob2']=data[str(year-i)+', '+'Выручка']/data[str(year-i)+', '+'Дебиторская задолженность'] # Коэффициент оборачиваемости дебиторской задолженности
        df_new[str(i)+', '+'cf_ob3']=data[str(year-i)+', '+'Выручка']/data[str(year-i)+', '+'Кредиторская задолженность'] # Коэффициент оборачиваемости кредиторской задолженности
        df_new[str(i)+', '+'cf_ob4']=data[str(year-i)+', '+'Выручка']/data[str(year-i)+', '+'Капитал и резервы'] # Коэффициент оборачиваемости собственного капитала                    
        df_new[str(i)+', '+'cf_ob5']=data[str(year-i)+', '+'Оборотные активы']/data[str(year-i)+', '+'Активы'] # Доля оборотных средств в активах
        ### 3. Коэффициенты рентабельности
        df_new[str(i)+', '+'cf_rent1']=data[str(year-i)+', '+'Прибыль (убыток) от продажи']/data[str(year-i)+', '+'Выручка'] # Рентабельность продаж
        df_new[str(i)+', '+'cf_rent2']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/data[str(year-i)+', '+'Себестоимость продаж'] # Рентабельность затрат
        df_new[str(i)+', '+'cf_rent3']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/data[str(year-i)+', '+'Активы'] # Экономическая рентабельность  
        df_new[str(i)+', '+'сf_rent4']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/data[str(year-i)+', '+'Капитал и резервы'] # Рентабельность собственного капитала
        df_new[str(i)+', '+'сf_rent5']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/data[str(year-i)+', '+'Основные средства'] # Рентабельность основных средств
        df_new[str(i)+', '+'сf_rent6']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/(
            data[str(year-i)+', '+'Заёмные средства (долгосрочные)']+data[str(year-i)+', '+'Заёмные средства (краткосрочные)']) # Рентабельность заёмного капитала
        df_new[str(i)+', '+'cf_rent7']=data[str(year-i)+', '+'Прибыль (убыток) до налогообложения']/data[str(year-i)+', '+'Оборотные активы'] # Рентабельность оборотных средств
        ### 4. Коэффициенты рыночной устойчивости
        df_new[str(i)+', '+'cf_ru1']=data[str(year-i)+', '+'Собственные оборотные средства']/data[str(year-i)+', '+'Капитал и резервы'] # Коэффициент маневренности
        df_new[str(i)+', '+'cf_ru2']=(data[str(year-i)+', '+'Краткосрочные обязательства']+data[str(year-i)+', '+'Долгосрочные обязательства'])/(
                          data[str(year-i)+', '+'Капитал и резервы'])                                      # Коэффициент капитализации
        df_new[str(i)+', '+'cf_ru3']=data[str(year-i)+', '+'Собственные оборотные средства']/data[str(year-i)+', '+'Оборотные активы'] # Коэффициент обеспеченности собственными средствами
        df_new[str(i)+', '+'cf_ru4']=data[str(year-i)+', '+'Капитал и резервы']/data[str(year-i)+', '+'Заёмные средства'] # Коэффициент финансирования
        df_new[str(i)+', '+'cf_ru5']=data[str(year-i)+', '+'Дебиторская задолженность']/data[str(year-i)+', '+'Кредиторская задолженность'] # Коэффициент дебиторской/кредиторской задолженности
        df_new[str(i)+', '+'cf_ru6']=data[str(year-i)+', '+'Краткосрочные обязательства']/(
                         data[str(year-i)+', '+'Краткосрочные обязательства']+data[str(year-i)+', '+'Долгосрочные обязательства']) # Коэффициент краткосрочной задолженности
        df_new[str(i)+', '+'cf_ru7']=data[str(year-i)+', '+'Долгосрочные обязательства']/data[str(year-i)+', '+'Внеоборотные активы'] # Коэффициент структуры долгосрочных вложений
        df_new[str(i)+', '+'cf_ru8']=data[str(year-i)+', '+'Долгосрочные обязательства']/(
                        data[str(year-i)+', '+'Долгосрочные обязательства']+data[str(year-i)+', '+'Капитал и резервы']) # Коэффициент привлечения средств
        ### 5. Коэффициенты финансовой устойчивости
        df_new[str(i)+', '+'cf_fu1']=(data[str(year-i)+', '+'Капитал и резервы']+data[str(year-i)+', '+'Заёмные средства (долгосрочные)'])/data[str(year-i)+', '+'Активы'] # Коэффициент финансовой устойчивости
        df_new[str(i)+', '+'cf_fu2']=data[str(year-i)+', '+'Капитал и резервы']/data[str(year-i)+', '+'Активы'] # Коэффициент финансовой независимости
        df_new[str(i)+', '+'cf_fu3']=data[str(year-i)+', '+'Заёмные средства']/data[str(year-i)+', '+'Капитал и резервы'] # Коэффициент соотношения заемных и собственных средств
        df_new[str(i)+', '+'cf_fu4']=data[str(year-i)+', '+'Заёмные средства']/data[str(year-i)+', '+'Активы'] # Коэффициент концентрации заемного капитала
        df_new[str(i)+', '+'cf_fu5']=(data[str(year-i)+', '+'Капитал и резервы']-data[str(year-i)+', '+'Внеоборотные активы'])/data[str(year-i)+', '+'Оборотные активы'] # Коэффициент обеспеченности оборотных активов собственными оборотными средствами
        df_new[str(i)+', '+'cf_fu6']=data[str(year-i)+', '+'Собственные оборотные средства']/data[str(year-i)+', '+'Оборотные активы'] # Коэффициент обеспеченности собственными источниками финансирования
    
    return df_new

data19_new=feature_generation(data19,2019)
data20_new=feature_generation(data20,2020)

#### Генерируем приросты коэффициентов

In [7]:
def coef_growth_generation(data):
    df_new=data.copy()
    # Берём сгенеренные фичи
    all_feats=[c for c in data.columns if c not in ('ID','ПДЗ','target')]
    # Фичи каждого года
    feats_1=[c for c in all_feats if c.split(',')[0]=='1']
    feats_2=[c for c in all_feats if c.split(',')[0]=='2']
    feats_3=[c for c in all_feats if c.split(',')[0]=='3']
    # Генерим динамику
    for i in range(len(feats_1)):
        df_new[f'1-2,{feats_1[i].split(",")[1]}']=df_new[feats_2[i]]-df_new[feats_1[i]]
        df_new[f'2-3,{feats_1[i].split(",")[1]}']=df_new[feats_3[i]]-df_new[feats_2[i]]
        
    return df_new

data19_new=coef_growth_generation(data19_new)
data20_new=coef_growth_generation(data20_new)

#### Добавляем таргет

In [8]:
data19_new['ПДЗ']=data19['Макс. ПДЗ за 2019 год, дней']
data20_new['ПДЗ']=data20['Макс. ПДЗ за 2020 год, дней']

#### Преобразуем таргет

In [9]:
def make_target_multi(data):
    data['target'] = np.where(data['ПДЗ']==0,0,
                                  np.where((data['ПДЗ']>0)&(data['ПДЗ']<=30),1,
                                  np.where((data['ПДЗ']>30)&(data['ПДЗ']<=60),2,
                                  np.where((data['ПДЗ']>60)&(data['ПДЗ']<=90),3,4))))
    
    return data

data19_new=make_target_multi(data19_new)
data20_new=make_target_multi(data20_new)

#### Преобразуем пропуски и бесконечности

In [10]:
def replace_na_inf(data):
    data=data.replace(np.inf,100)
    data=data.replace(-np.inf,-100)
    data=data.fillna(0)
    return(data)
 
data19_new=replace_na_inf(data19_new)
data20_new=replace_na_inf(data20_new)

#### Сохраняем в csv

In [11]:
data19_new.to_csv('df2019_model.csv')
data20_new.to_csv('df2020_model.csv')